In [2]:
%pip install scikit-learn
%pip install matplotlib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [4]:
# Load the pickle file
print("Loading datafile...")
## 경로 바꾸기
with open("./mon_standard.pkl", 'rb') as fi: # Path to mon_standard.pkl in local
    mon_data = pickle.load(fi)

with open('./unmon_standard10.pkl', 'rb') as f1:  # Path to unmon_standard10.pkl in local
    unmon_data = pickle.load(f1)


Loading datafile...


In [5]:
USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

mon_X1 = [] # Array to store instances (timestamps) - 19,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
mon_X2 = [] # Array to store instances (direction*size) - size information
mon_y = [] # Array to store the site of each instance - 19,000 instances, e.g., [0, 0, 0, 0, 0, 0, ..., 94, 94, 94, 94, 94]


# Differentiate instances and sites, and store them in the respective x and y arrays
# x array (direction*timestamp), y array (site label)
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE # Calculate which site's URL the current URL being processed belongs to and set that value as the label. Thus, URLs fetched from the same site are labeled identically.
    for sample in mon_data[i]:
        size_seq = []
        time_seq = []
        for c in sample:
            dr = 1 if c > 0 else -1
            time_seq.append(abs(c))
            size_seq.append(dr * 512)
        # print(len(time_seq))
        mon_X1.append(time_seq)
        mon_X2.append(size_seq)
        mon_y.append(label)
mon_size = len(mon_y)

print(f'Total Mon samples: {mon_size}') # Output: 19000

UNMON_TOTAL_URLS = 10000
unmon_X1 = [] # Array to store instances (timestamps) - 10,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
unmon_X2 = [] # Array to store instances (direction*size) - size information

for i in range(UNMON_TOTAL_URLS):
    size_seq = []
    time_seq = []
    for c in unmon_data[i]:
        dr = 1 if c > 0 else -1
        time_seq.append(abs(c))
        size_seq.append(dr * 512) # In the pickle file, there is no size information, so the conversion code is set to multiply by 512 uniformly.
    unmon_X1.append(time_seq)
    unmon_X2.append(size_seq)
unmon_size = len(unmon_X1)
unmon_y = [ 95 for sample_idx in range(unmon_size)]
print(f'Total Unmon samples: {unmon_size}') # Output: 19000

Total Mon samples: 19000
Total Unmon samples: 10000


In [6]:
X1 = mon_X1 + unmon_X1 
X1 = np.array(X1,dtype=object)
X2 = mon_X2 + unmon_X2
X2 = np.array(X2,dtype=object)
# feature기반으로 test data생성할 때 넣는 y값이 다름
y_multi = mon_y + unmon_y
y_binary = [1 for sample_idx in range(mon_size)] + unmon_y
y_multi = np.array(y_multi, dtype=object)
y_binary = np.array(y_binary, dtype=object)


print(f"Total data X1 feature shape: {X1.shape}")
print(f"Total data X2 shape: {X2.shape}")
print(f"Total data y_multi shape: {y_multi.shape}")
print(f"Total data y_binary shape: {y_binary.shape}")

Total data X1 feature shape: (29000,)
Total data X2 shape: (29000,)
Total data y_multi shape: (29000,)
Total data y_binary shape: (29000,)


In [7]:
# 1. number of incoming packets
incoming_packets = [np.sum(np.array(sub_array) < 0) for sub_array in X2]
incoming_packets = np.array(incoming_packets).reshape(-1, 1)

# 2. the total number of incoming packets stats in first 30 packets
incoming_packets_in_first_30_packets = [np.sum(np.array(sub_array[:30]) < 0) for sub_array in X2]
incoming_packets_in_first_30_packets = np.array(incoming_packets_in_first_30_packets).reshape(-1, 1)

# 3. the total number of outcoming packets stats in first 30 packets
outgoing_packets_in_first_30_packets = [np.sum(np.array(sub_array[:30]) > 0) for sub_array in X2]
outgoing_packets_in_first_30_packets = np.array(outgoing_packets_in_first_30_packets).reshape(-1, 1)

# 4. number of outgoing packets as a fraction of the total number of packets
outgoing_fraction = [np.sum(np.array(sub_array) > 0) / len(sub_array) if len(sub_array) != 0 else 0 for sub_array in X2]
outgoing_fraction = np.array(outgoing_fraction).reshape(-1, 1)

# 5. total number of packets
total_packets_count = [len(sub_array) for sub_array in X2]
total_packets_count = np.array(total_packets_count)
total_packets_count_2D = total_packets_count.reshape(-1, 1)

# 6. Compute fraction of incoming packets for each entry in X2
incoming_fraction = [np.sum(np.array(sub_array) < 0) / len(sub_array) if len(sub_array) != 0 else 0 for sub_array in X2]
incoming_fraction = np.array(incoming_fraction).reshape(-1, 1)

# 7. number of outgoing packets
outgoing_packets = [np.sum(np.array(sub_array) > 0) for sub_array in X2]
outgoing_packets = np.array(outgoing_packets).reshape(-1, 1)

# 8. standard deviation of the outgoing packet ordering list
std_deviation_outgoing = [np.std(sub_array) for sub_array in X2]
std_deviation_outgoing = np.array(std_deviation_outgoing).reshape(-1, 1)

# 9. 
avg_outgoing_order = []
for time_seq, size_seq in zip(X1, X2):
    outgoing_times = [t for t, s in zip(time_seq, size_seq) if s > 0]
    avg_outgoing_order.append(np.mean(outgoing_times) if outgoing_times else 0)
avg_outgoing_order = np.array(avg_outgoing_order).reshape(-1, 1)


# 10.
incoming_counts = []
outgoing_counts = []
total_counts = []
combined_counts = []

for size_seq in X2:
    incoming_counts.append(np.sum(np.array(size_seq) < 0))
    outgoing_counts.append(np.sum(np.array(size_seq) > 0))
    total_counts.append(len(size_seq))
total_counts = np.array(total_counts).reshape(-1, 1)

# 11. Sum of incoming, outgoing and total number of packets
for i in range(len(total_counts)):
    combined_count = incoming_counts[i] + outgoing_counts[i] + total_counts[i]
    combined_counts.append(combined_count)
combined_counts = np.array(combined_counts).reshape(-1, 1)


# 12.
packet_concentration = []

for time_seq in X1:
    # Packet Concentration: Calculate time differences between packets
    if len(time_seq) > 1:
        time_diffs = np.diff(time_seq)
        concentration_feature = np.mean(time_diffs)  # 평균 시간 간격
    else:
        concentration_feature = 0  # 패킷이 하나 뿐인 경우

    packet_concentration.append(concentration_feature)
packet_concentration = np.array(packet_concentration).reshape(-1, 1)

print(f"1. feature shape : {incoming_packets.shape}")
print(f"2. feature shape : {incoming_packets_in_first_30_packets.shape}")
print(f"3. feature shape : {outgoing_packets_in_first_30_packets.shape}")
print(f"4. feature shape : {outgoing_fraction.shape}")
print(f"5. feature shape : {total_packets_count_2D.shape}")
print(f"6. feature shape : {incoming_fraction.shape}")
print(f"7. feature shape : {outgoing_packets.shape}")
print(f"8. feature shape : {std_deviation_outgoing.shape}")
print(f"9. feature shape : {avg_outgoing_order.shape}")
print(f"10. feature shape : {total_counts.shape}")
print(f"11. feature shape : {combined_counts.shape}")
print(f"12. feature shape : {packet_concentration.shape}")


1. feature shape : (29000, 1)
2. feature shape : (29000, 1)
3. feature shape : (29000, 1)
4. feature shape : (29000, 1)
5. feature shape : (29000, 1)
6. feature shape : (29000, 1)
7. feature shape : (29000, 1)
8. feature shape : (29000, 1)
9. feature shape : (29000, 1)
10. feature shape : (29000, 1)
11. feature shape : (29000, 1)
12. feature shape : (29000, 1)


In [10]:
X_combined = np.hstack((
    incoming_packets,
    incoming_packets_in_first_30_packets,
    outgoing_packets_in_first_30_packets,
    outgoing_fraction,
    total_packets_count_2D,
    incoming_fraction,
    outgoing_packets,
    std_deviation_outgoing,
    avg_outgoing_order,
    total_counts,
    combined_counts,
    packet_concentration
)).astype(np.float64) 

label_encoder = LabelEncoder()
# for binary test and multi test
y_binary_test = label_encoder.fit_transform(y_binary)
y_multi_test = label_encoder.fit_transform(y_multi)



In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Open World - binary class
## Naive bayes

In [12]:
# Assuming X_categorical and y are your features and labels respectively
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_binary_test, test_size=0.3, random_state=42)

# Initialize the Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Train the classifier
gnb.fit(X_train, y_train)

# Make predictions
y_pred = gnb.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Generate a classification report
#print("\nClassification Report:\n")
#print(classification_report(y_test, y_pred, zero_division=0))

# Confusion Matrix
#conf_matrix = confusion_matrix(y_test, y_pred)

# Plotting the Confusion Matrix
#plt.figure(figsize=(10, 8))
#sns.heatmap(conf_matrix, annot=True, fmt='g')
#plt.title('Confusion Matrix')
#plt.ylabel('Actual Labels')
#plt.xlabel('Predicted Labels')
#plt.show()

Accuracy: 0.7260


## SVM

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [14]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_binary_test, test_size=0.3, random_state=42)

# Initializing the Support Vector Machine (SVM) model
svm_model = SVC()

# Training the model with the training data
svm_model.fit(X_train, y_train)

# Making predictions on the test data
y_pred = svm_model.predict(X_test)

# Calculating the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')


Accuracy: 0.6574


# Open World - multi class
## Naive bayes

In [15]:
# Assuming X_categorical and y are your features and labels respectively
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_multi_test, test_size=0.3, random_state=42)

# Initialize the Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Train the classifier
gnb.fit(X_train, y_train)

# Make predictions
y_pred = gnb.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')


Accuracy: 0.1505


## SVM

In [16]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_multi_test, test_size=0.3, random_state=42)

# Initializing the Support Vector Machine (SVM) model
svm_model = SVC()

# Training the model with the training data
svm_model.fit(X_train, y_train)

# Making predictions on the test data
y_pred = svm_model.predict(X_test)

# Calculating the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.3426


# Hyper parameter Tuning SVM 1st trial

In [17]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [19]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_multi_test, test_size=0.3, random_state=42)

# Initializing the Support Vector Machine (SVM) model
svm_model = SVC(kernel='rbf', decision_function_shape='ovo')

# 하이퍼파라미터 범위 축소
param_grid = {
    'C': [1, 10, 100],
    'gamma': [0.01, 0.1, 1.0],
    'kernel': ['rbf']
}

# 랜덤 서치 초기화
random_search = GridSearchCV(SVC(decision_function_shape='ovo'), param_grid, cv=3, scoring='accuracy', verbose=2)

# 모델 훈련
random_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", random_search.best_params_)

# 최적의 모델로 예측
best_svm_model = random_search.best_estimator_
y_pred = best_svm_model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END ........................C=1, gamma=0.01, kernel=rbf; total time= 2.6min
[CV] END ........................C=1, gamma=0.01, kernel=rbf; total time= 2.6min
[CV] END ........................C=1, gamma=0.01, kernel=rbf; total time= 2.6min
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time= 2.6min
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time= 2.6min
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .........................C=1, gamma=1.0, kernel=rbf; total time= 2.6min
[CV] END .........................C=1, gamma=1.0, kernel=rbf; total time= 2.6min
[CV] END .........................C=1, gamma=1.0, kernel=rbf; total time= 2.5min
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time= 2.6min
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time= 2.7min
[CV] END .......................C=10, gamma=0.01,

# Hyper parameter Tuning SVM 2nd trial

In [20]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_multi_test, test_size=0.3, random_state=42)

# Initializing the Support Vector Machine (SVM) model
svm_model = SVC(kernel='rbf', decision_function_shape='ovo')

param_grid = {
    'C': [100, 200, 300],
    'gamma': [0.1],
    'kernel': ['rbf']
}

# 그리드 서치 초기화
grid_search = GridSearchCV(SVC(decision_function_shape='ovo'), param_grid, cv=3, scoring='accuracy', verbose=2)

# 모델 훈련
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)

# 최적의 모델로 예측
best_svm_model = grid_search.best_estimator_
y_pred = best_svm_model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time= 2.6min
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .......................C=200, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .......................C=200, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .......................C=200, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .......................C=300, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .......................C=300, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .......................C=300, gamma=0.1, kernel=rbf; total time= 2.8min
Best Hyperparameters: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy: 0.3793


# Hyper parameter Tuning SVM 3rd trial

In [21]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_multi_test, test_size=0.3, random_state=42)

# Initializing the Support Vector Machine (SVM) model
svm_model = SVC(kernel='rbf', decision_function_shape='ovo')

param_grid = {
    'C': [10, 20, 30],
    'gamma': [0.1],
    'kernel': ['rbf']
}

# 그리드 서치 초기화
grid_search = GridSearchCV(SVC(decision_function_shape='ovo'), param_grid, cv=3, scoring='accuracy', verbose=2)

# 모델 훈련
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)

# 최적의 모델로 예측
best_svm_model = grid_search.best_estimator_
y_pred = best_svm_model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ........................C=20, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ........................C=20, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ........................C=20, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ........................C=30, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ........................C=30, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ........................C=30, gamma=0.1, kernel=rbf; total time= 2.8min
Best Hyperparameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy: 0.3792


# Hyper parameter Tuning SVM 4th trial

In [22]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_multi_test, test_size=0.3, random_state=42)

# Initializing the Support Vector Machine (SVM) model
svm_model = SVC(kernel='rbf', decision_function_shape='ovo')

param_grid = {
    'C': [1, 5, 10],
    'gamma': [0.1],
    'kernel': ['rbf']
}

# 그리드 서치 초기화
grid_search = GridSearchCV(SVC(decision_function_shape='ovo'), param_grid, cv=3, scoring='accuracy', verbose=2)

# 모델 훈련
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)

# 최적의 모델로 예측
best_svm_model = grid_search.best_estimator_
y_pred = best_svm_model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time= 2.6min
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time= 2.6min
[CV] END .........................C=5, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .........................C=5, gamma=0.1, kernel=rbf; total time= 2.8min
[CV] END .........................C=5, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time= 2.8min
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time= 2.7min
Best Hyperparameters: {'C': 5, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy: 0.3790


# Hyper parameter Tuning SVM 5th trial

In [23]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_multi_test, test_size=0.3, random_state=42)

# Initializing the Support Vector Machine (SVM) model
svm_model = SVC(kernel='rbf', decision_function_shape='ovo')

param_grid = {
    'C': [300, 500, 1000],
    'gamma': [0.1],
    'kernel': ['rbf']
}

# 그리드 서치 초기화
grid_search = GridSearchCV(SVC(decision_function_shape='ovo'), param_grid, cv=3, scoring='accuracy', verbose=2)

# 모델 훈련
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)

# 최적의 모델로 예측
best_svm_model = grid_search.best_estimator_
y_pred = best_svm_model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END .......................C=300, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .......................C=300, gamma=0.1, kernel=rbf; total time= 2.8min
[CV] END .......................C=300, gamma=0.1, kernel=rbf; total time= 2.8min
[CV] END .......................C=500, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .......................C=500, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END .......................C=500, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ......................C=1000, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ......................C=1000, gamma=0.1, kernel=rbf; total time= 2.7min
[CV] END ......................C=1000, gamma=0.1, kernel=rbf; total time= 2.7min
Best Hyperparameters: {'C': 300, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy: 0.3793
